In [92]:
print("""
@Description: Machine learning for classification
@Author(s): Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime: 2023-07-14 10:34:33
""")


@Description: Machine learning for classification
@Author(s): Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime: 2023-07-14 10:34:33



In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

According to the description, this dataset has the following information:
- Services of the customers: phone; multiple lines; internet; tech support and extra services such as online security, backup, device protection, and TV streaming
- Account information: how long they have been clients, type of contract, type of payment method
- Charges: how much the client was charged in the past month and in total
- Demographic information: gender, age, and whether they have dependents or a partner
- Churn: yes/no, whether the customer left the company within the past month

In [94]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
# 有些时候会压缩数据以减少内存使用
# categorical_mask = (df.nunique() < 5) & (df.dtypes == 'object')

In [95]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [96]:
# 转置后可以在屏幕上看到更多的数据(columns)
df.sample(2).T

,5736,4592
customerID,8749-CLJXC,1897-RCFUM
gender,Male,Female
SeniorCitizen,0,0
Partner,No,Yes
Dependents,No,Yes
tenure,1,39
PhoneService,Yes,Yes
MultipleLines,No,Yes
InternetService,No,No
OnlineSecurity,No internet service,No internet service


In [97]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [98]:
# errors='coerce'会自动将缺失转化为NaN
total_charges = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [99]:
# 缺失存在TotalCharges为空字符串的样本行
df[total_charges.isnull()][['customerID', 'TotalCharges']]

,customerID,TotalCharges
488,4472-LVYGI,
753,3115-CZMZD,
936,5709-LVOEQ,
1082,4367-NUYAO,
1340,1371-DWPAZ,
3331,7644-OMVMY,
3826,3213-VVOLG,
4380,2520-SGTTA,
5218,2923-ARZLG,
6670,4075-WKNIU,


In [100]:
# 使用0值填充缺失值
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce').fillna(0)

In [101]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
string_columns = list(df.dtypes[df.dtypes == 'object'].index)
for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [102]:
# For binary classification, all models typically 
# expect a number: 0 for “no” and 1 for “yes.” 
df['churn'] = (df['churn'] == 'yes').astype(int)

In [103]:
from sklearn.model_selection import train_test_split

In [109]:
# 有进行分层抽样吗？
df_train_full, df_test = train_test_split(df, test_size=.2, 
                                          random_state=1)
# df_train_full, df_test = train_test_split(df, test_size=.2, 
#                                           stratify=df['churn'],
#                                           random_state=1)
df_train, df_val = train_test_split(df_train_full, test_size=.33, random_state=1)
y_train = df_train['churn'].values
y_val = df_val['churn'].values
del df_train['churn']
del df_val['churn']

In [114]:
# df_train_full.isnull().any().any()
# False表示没有缺失值
# We should always check for any missing values in the dataset 
# because many machine learning models cannot easily deal with missing data.
df_train_full.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [115]:
# Another thing we should do is check the distribution of values in the target variable.
df_train_full['churn'].value_counts()
# df_train_full['churn'].value_counts(normalize=True)

churn
0    4113
1    1521
Name: count, dtype: int64

In [137]:
df_train_full['churn'].value_counts(normalize=True)[1]
global_mean = df_train_full['churn'].mean()
round(global_mean, 2)

0.27

In [125]:
global_mean

0.26996805111821087

In [142]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [143]:
df_train_full[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

### Feature importance

Knowing how other variables affect the target variable, churn, is the key to understanding the data and building a good model. This process is called feature importance analysis, and it’s often done as a part of exploratory data analysis to figure out which variables will be useful for the model. It also gives us additional insights about the dataset and helps answer questions like “What makes customers churn?” and “What are the characteristics of people who churn?”

We have two different kinds of features: categorical and numerical. Each kind has different ways of measuring feature importance, so we will look at each separately.

#### churn ratio

In [155]:
female_mask = df_train_full['gender'] == 'female'
female_mean = df_train_full[female_mask]['churn'].mean()
male_mean = df_train_full[~female_mask]['churn'].mean()
# df_train_full.groupby('gender')['churn'].mean().to_list()
female_mean, male_mean, global_mean

(0.27682403433476394, 0.2632135306553911, 0.26996805111821087)

In [168]:
partner_mask = df_train_full['partner'] == 'yes'
partner_yes = df_train_full[partner_mask]['churn'].mean()
partner_no = df_train_full[~partner_mask]['churn'].mean()
# df_train_full.groupby('partner')['churn'].mean()
partner_yes, partner_no, global_mean

(0.20503330866025166, 0.3298090040927694, 0.26996805111821087)

#### risk ratio

\begin{equation}
risk=\frac{group~rate}{global~rate}
\end{equation}

\begin{equation}
risk = \frac{negative~outcome~rate~in~group~1}{negative~outcome~rate~in~group~2}
\end{equation}

In [174]:
global_mean = df_train_full['churn'].mean()
df_group = df_train_full.groupby(by=['gender'])['churn'].agg(['mean'])
df_group['diff'] = df_group['mean'] - global_mean
df_group['risk'] = df_group['mean'] / global_mean
df_group

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


In [178]:
# do that for all categorical variables
from IPython.display import display
for col in categorical:
    df_group = df_train_full.groupby(by=col)['churn'].agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_mean
    df_group['risk'] = df_group['mean']/global_mean
    # print(df_group)
    display(df_group)

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


,mean,diff,risk
seniorcitizen,,,
0,0.242270,-0.027698,0.897403
1,0.413377,0.143409,1.531208


,mean,diff,risk
partner,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


,mean,diff,risk
dependents,,,
no,0.313760,0.043792,1.162212
yes,0.165666,-0.104302,0.613651


,mean,diff,risk
phoneservice,,,
no,0.241316,-0.028652,0.893870
yes,0.273049,0.003081,1.011412


...

,mean,diff,risk
streamingtv,,,
no,0.342832,0.072864,1.269897
no_internet_service,0.077805,-0.192163,0.288201
yes,0.302723,0.032755,1.121328


,mean,diff,risk
streamingmovies,,,
no,0.338906,0.068938,1.255358
no_internet_service,0.077805,-0.192163,0.288201
yes,0.307273,0.037305,1.138182


,mean,diff,risk
contract,,,
month-to-month,0.431701,0.161733,1.599082
one_year,0.120573,-0.149395,0.446621
two_year,0.028274,-0.241694,0.104730


,mean,diff,risk
paperlessbilling,,,
no,0.172071,-0.097897,0.637375
yes,0.338151,0.068183,1.252560


,mean,diff,risk
paymentmethod,,,
bank_transfer_(automatic),0.168171,-0.101797,0.622928
credit_card_(automatic),0.164339,-0.105630,0.608733
electronic_check,0.455890,0.185922,1.688682
mailed_check,0.193870,-0.076098,0.718121
